In [14]:
library(sp)
library(rgdal)
library(raster)

In [15]:
PISCO.file <- "D://PISCO/PPISCOpd.nc"

In [16]:
Huarpa.stations<-read.csv('D:/Proyectos_GitHub/curso_hidrologia/data/Estaciones/Huarpa.estaciones.csv',sep=';')
Huarpa.stations

Estacion,x,y,Alt
<chr>,<dbl>,<dbl>,<int>
San Miguel,-73.98333,-13.01667,3323
Hacienda Cochas,-73.88333,-13.03333,3323
La Quinua,-74.14139,-13.05528,3240
Wayllapampa,-74.21667,-13.07639,2470
Huamanga,-74.23694,-13.15000,2761
Allpachaca,-74.26667,-13.38333,3600


## Creamos una función auxiliar para crear un dataframe con información espacial de la ubicación de las estaciones

In [17]:
points_wgs84=function(data){
  coordinates(data) <- ~x+y
  proj4string(data)='+proj=longlat +datum=WGS84 +no_defs +ellps=WGS84 +towgs84=0,0,0'
  data
}

In [18]:
estaciones=points_wgs84(Huarpa.stations)

## Cargamos la información raster

In [9]:
pisco_raster=stack(PISCO.file)
pisco_raster=setZ(pisco_raster, seq(as.Date("1981-01-01"), as.Date("2016-12-31"), by = "day"))

Loading required namespace: ncdf4



## Extraemos los valores del stack de raster en las ubicaciones de las estaciones

In [50]:
data_diaria <-data.frame(t(round(extract(pisco_raster, estaciones),2)))
names(data_diaria) <- Huarpa.stations$Estacion

In [51]:
date=seq(as.Date("1981-01-01"), as.Date("2016-12-31"), by = "day")
data_diaria = cbind(date,data_diaria)

In [68]:
head(data_diaria,10)

,date,San Miguel,Hacienda Cochas,La Quinua,Wayllapampa,Huamanga,Allpachaca
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
X0,1981-01-01,1.58,2.13,1.80,0.96,0.78,0.00
X1,1981-01-02,0.00,0.00,0.86,3.12,2.72,0.55
X2,1981-01-03,3.29,4.38,3.07,1.14,0.89,0.00
X3,1981-01-04,0.00,0.00,0.00,0.00,0.85,1.58
X4,1981-01-05,2.38,3.22,4.08,6.90,6.37,2.74
X5,1981-01-06,1.71,4.35,3.04,9.09,4.88,3.75
X6,1981-01-07,0.13,0.17,0.02,0.01,0.04,0.38
X7,1981-01-08,0.02,0.00,0.04,0.02,0.08,0.09
X8,1981-01-09,0.35,0.00,8.77,7.87,13.52,1.55


## Para obtener datos mensuales tenemos que realizar una suma entre los dias de cada mes por cada año

In [69]:
data_mensual=rowsum(data_diaria[,-1], format(data_diaria$date, "%m-%Y"),reorder = F )
data_mensual=cbind(fecha=row.names(data_mensual),data.frame(data_mensual,row.names = NULL))

In [70]:
head(data_mensual,10)

,fecha,San.Miguel,Hacienda.Cochas,La.Quinua,Wayllapampa,Huamanga,Allpachaca
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,01-1981,41.76,55.06,79.56,101.94,90.83,44.64
2,02-1981,66.56,78.13,124.63,205.45,185.73,140.60
3,03-1981,54.86,68.75,63.36,78.20,64.65,51.55
4,04-1981,10.66,9.53,13.60,16.83,17.09,11.91
5,05-1981,6.88,4.28,12.74,14.21,6.96,2.54
6,06-1981,1.42,1.09,5.33,7.16,7.93,1.80
7,07-1981,0.30,0.28,1.24,2.28,1.76,0.68
8,08-1981,1.66,2.32,0.40,0.44,2.18,8.33
9,09-1981,11.82,8.76,30.89,40.93,27.57,11.39


## Guardamos nuestros resultados en archivos csv

In [71]:
write.table(data_diaria,'D:/Proyectos_GitHub/curso_hidrologia/data/Estaciones/Huarpa_diario.csv',sep = ',',row.names = F)
write.table(data_mensual,'D:/Proyectos_GitHub/curso_hidrologia/data/Estaciones/Huarpa_mensual.csv',sep = ',',row.names = F)